In [471]:
import cv2
#globbing utility. upsamle discriminator
import glob
import os
import numpy as np
from tqdm import tqdm


# load, split and scale the maps dataset ready for training
from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
import random

# example of pix2pix gan for satellite to map image-to-image translation
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.utils import plot_model



In [472]:
d = './dataset/'
lst = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]

lst

['./dataset/a']

In [473]:
src = []
tgt = []
P = []
for path in tqdm(lst):
    path1 = path +"/side/*.png"
    folder = path.split('/')[-1]
    tgtpath = path+"/"+folder+"_01.png"
    #print(tgtpath)
    for file in glob.glob(path1):
        #print (file)
        a= cv2.imread(file)
        a= cv2.resize(a, (256,256), interpolation = cv2.INTER_AREA)
        src.append(a)
        b= cv2.imread(tgtpath)
        b= cv2.resize(b, (256,256), interpolation = cv2.INTER_AREA)
        tgt.append(b)
        

print(folder)
print(tgtpath)
print(path1)

100%|██████████| 1/1 [00:00<00:00, 63.95it/s]

a
./dataset/a/a_01.png
./dataset/a/side/*.png


In [474]:
src = np.asarray(src)
tgt = np.asarray(tgt)

print(src.shape)
print(tgt.shape)

(4, 256, 256, 3)
(4, 256, 256, 3)


In [475]:
filename = 'multi.npz'
savez_compressed(filename, src, tgt)
print('Saved dataset: ', filename)

Saved dataset:  multi.npz


In [476]:
data = load('multi.npz')
src_images, tar_images = data['arr_0'], data['arr_1']

In [477]:
# load the prepared dataset
from numpy import load
from matplotlib import pyplot
# load the dataset
data = load('multi.npz')
src_images, tar_images = data['arr_0'], data['arr_1']
print('Loaded: ', src_images.shape, tar_images.shape)
# # plot source images
# n_samples = 3
# r = random.randint(0,len(src_images))
# for i in range(n_samples):
# 	pyplot.subplot(2, n_samples, 1 + i)
# 	pyplot.axis('off')
# 	pyplot.imshow(src_images[r+i].astype('uint8'))
# # plot target image
# for i in range(n_samples):
# 	pyplot.subplot(2, n_samples, 1 + n_samples + i)
# 	pyplot.axis('off')
# 	pyplot.imshow(tar_images[r+i].astype('uint8'))
# pyplot.show()

Loaded:  (4, 256, 256, 3) (4, 256, 256, 3)


In [479]:
# define the discriminator model
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model
model = define_discriminator((256, 256, 3))
model.summary()

Model: "model_122"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_181 (InputLayer)          [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_182 (InputLayer)          [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
concatenate_295 (Concatenate)   (None, 256, 256, 6)  0           input_181[0][0]                  
                                                                 input_182[0][0]                  
__________________________________________________________________________________________________
conv2d_634 (Conv2D)             (None, 128, 128, 64) 6208        concatenate_295[0][0]    

c:\Users\probook\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [480]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, Flatten, Dense, Reshape, Conv2DTranspose

# def define_discriminator(image_shape):
#     # weight initialization
#     init = RandomNormal(stddev=0.02)
#     # source image input
#     in_src_image = Input(shape=image_shape)
#     # target image input
#     in_target_image = Input(shape=image_shape)
#     # concatenate images channel-wise
#     merged = Concatenate()([in_src_image, in_target_image])
#     # C64
#     d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
#     d = LeakyReLU(alpha=0.2)(d)
#     # C128
#     d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
#     d = BatchNormalization()(d)
#     d = LeakyReLU(alpha=0.2)(d)
#     # C256
#     d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
#     d = BatchNormalization()(d)
#     d = LeakyReLU(alpha=0.2)(d)
#     # C512
#     d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
#     d = BatchNormalization()(d)
#     d = LeakyReLU(alpha=0.2)(d)
#     # second last output layer
#     d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
#     d = BatchNormalization()(d)
#     d = LeakyReLU(alpha=0.2)(d)
#     # reshape output
#     d = Conv2DTranspose(256, (4,4), strides=(4,4), padding='same', kernel_initializer=init)(d)
#     d = Conv2DTranspose(128, (4,4), strides=(4,4), padding='same', kernel_initializer=init)(d)
#     # patch output
#     d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
#     patch_out = Activation('sigmoid')(d)
#     # define model
#     model = Model([in_src_image, in_target_image], patch_out)
#     # compile model
#     opt = Adam(lr=0.0002, beta_1=0.5)
#     model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
#     return model
# model = define_discriminator((256, 256, 3))
# model.summary()

In [481]:
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, BatchNormalization, Add , UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Reshape

# def build_discriminator(input_shape=(256, 256, 3)):
#     # Input layer
#     input_img = Input(shape=input_shape)

#     # Downsampling
#     d1 = Conv2D(64, kernel_size=4, strides=2, padding='same')(input_img)
#     d1 = LeakyReLU(alpha=0.2)(d1)

#     d2 = Conv2D(128, kernel_size=4, strides=2, padding='same')(d1)
#     d2 = BatchNormalization()(d2)
#     d2 = LeakyReLU(alpha=0.2)(d2)

#     d3 = Conv2D(256, kernel_size=4, strides=2, padding='same')(d2)
#     d3 = BatchNormalization()(d3)
#     d3 = LeakyReLU(alpha=0.2)(d3)

#     d4 = Conv2D(512, kernel_size=4, strides=2, padding='same')(d3)
#     d4 = BatchNormalization()(d4)
#     d4 = LeakyReLU(alpha=0.2)(d4)

#     # Upsampling
#     u1 = UpSampling2D(size=(2, 2))(d4)
#     u1 = Conv2D(256, kernel_size=4, strides=1, padding='same')(u1)
#     u1 = BatchNormalization()(u1)
#     u1 = LeakyReLU(alpha=0.2)(u1)

#     u2 = UpSampling2D(size=(2, 2))(u1)
#     u2 = Conv2D(128, kernel_size=4, strides=1, padding='same')(u2)
#     u2 = BatchNormalization()(u2)
#     u2 = LeakyReLU(alpha=0.2)(u2)

#     u3 = UpSampling2D(size=(2, 2))(u2)
#     u3 = Conv2D(64, kernel_size=4, strides=1, padding='same')(u3)
#     u3 = BatchNormalization()(u3)
#     u3 = LeakyReLU(alpha=0.2)(u3)

#     # Output layer
#     output = Conv2D(1, kernel_size=4, strides=1, padding='same')(u3)

#     return Model(input_img, output)


# model2 = build_discriminator(input_shape=(256, 256, 3))
# model2.summary()

In [482]:
d_model = define_discriminator((256, 256, 3))
d_model.summary()

Model: "model_123"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_183 (InputLayer)          [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_184 (InputLayer)          [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
concatenate_296 (Concatenate)   (None, 256, 256, 6)  0           input_183[0][0]                  
                                                                 input_184[0][0]                  
__________________________________________________________________________________________________
conv2d_640 (Conv2D)             (None, 128, 128, 64) 6208        concatenate_296[0][0]    

In [483]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g

# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add upsampling layer
	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g

# define the standalone generator model
def define_generator(image_shape=(256,256,3)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# encoder model
	e1 = define_encoder_block(in_image, 64, batchnorm=False)
	e2 = define_encoder_block(e1, 128)
	e3 = define_encoder_block(e2, 256)
	e4 = define_encoder_block(e3, 512)
	e5 = define_encoder_block(e4, 512)
	e6 = define_encoder_block(e5, 512)
	e7 = define_encoder_block(e6, 512)
	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
	b = Activation('relu')(b)
	# decoder model
	d1 = decoder_block(b, e7, 512)
	d2 = decoder_block(d1, e6, 512)
	d3 = decoder_block(d2, e5, 512)
	d4 = decoder_block(d3, e4, 512, dropout=False)
	d5 = decoder_block(d4, e3, 256, dropout=False)
	d6 = decoder_block(d5, e2, 128, dropout=False)
	d7 = decoder_block(d6, e1, 64, dropout=False)
	# output
	g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

In [496]:
g_model = define_generator((256, 256, 3))
# plot_model(g_model, to_file='gen_model.png', show_shapes=True)
# g_model.summary()

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [485]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Input
import keras.backend as K
from keras.layers import Lambda


def perceptual_loss(in_src, gen_out):

    # Define a pre-trained VGG16 model without the last few layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    base_model.trainable = False
    
    # Define a new model that outputs the feature maps of the VGG16 model
    modelvgg = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_conv3').output)
    
    # Freeze the weights of the VGG16 model
    for layer in modelvgg.layers:
        layer.trainable = False
        
    # Define the perceptual loss function
    def ploss(in_src, gen_out):
        in_src = (in_src + 1) * 127.5
        gen_out = (gen_out + 1) * 127.5
        in_src = Lambda(lambda x: (x - 120.0) / 255.0)(in_src)  # normalize input image for VGG16
        gen_out = Lambda(lambda x: (x - 120.0) / 255.0)(gen_out)
        feature_true = modelvgg(in_src)
        feature_pred = modelvgg(gen_out)
        loss = K.mean(K.square(feature_true - feature_pred))
        return loss
    
    return ploss(in_src, gen_out)


In [486]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=[perceptual_loss , 'binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
	return model

    

In [487]:
model = define_gan(g_model , d_model, (256, 256, 3))
model.summary()

Model: "model_125"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_186 (InputLayer)          [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
model_124 (Functional)          (None, 256, 256, 3)  54429315    input_186[0][0]                  
__________________________________________________________________________________________________
model_123 (Functional)          (None, 16, 16, 1)    6968257     input_186[0][0]                  
                                                                 model_124[0][0]                  
Total params: 61,397,572
Trainable params: 54,419,459
Non-trainable params: 6,978,113
__________________________________________________________________________________________________


In [488]:
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 255.5) / 255.5
	X2 = (X2 - 255.5) / 255.5
	return [X1, X2]

In [489]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

In [490]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [491]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=3):
	# select a sample of input images
	[X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
	# generate a batch of fake samples
	X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
	# scale all pixels from [-1,1] to [0,1]
	X_realA = (X_realA + 1) / 2.0
	X_realB = (X_realB + 1) / 2.0
	X_fakeB = (X_fakeB + 1) / 2.0
	# plot real source images
	for i in range(n_samples):
		pyplot.subplot(3, n_samples, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(X_realA[i])
	# plot generated target image
	for i in range(n_samples):
		pyplot.subplot(3, n_samples, 1 + n_samples + i)
		pyplot.axis('off')
		pyplot.imshow(X_fakeB[i])
	# plot real target image
	for i in range(n_samples):
		pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
		pyplot.axis('off')
		pyplot.imshow(X_realB[i])
	# save plot to file
	filename1 = 'plot_%06d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%06d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

In [492]:
# train pix2pix model
def train(d_model, g_model, gan_model, base_model, dataset, n_epochs=2, n_batch=1):
	# determine the output square shape of the discriminator
	n_patch = d_model.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
    
    # Freeze the weights of the VGG16 model
   
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		# update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real,X_realB])
		# summarize performance
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
		# summarize model performance
		if (i+1) % (bat_per_epo * 10) == 0:
			summarize_performance(i, g_model, dataset)

In [493]:
# load image data
dataset = load_real_samples('multi.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)

Loaded (4, 256, 256, 3) (4, 256, 256, 3)


In [494]:
dataset[0].shape[1:]

(256, 256, 3)

In [495]:
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)

# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)
# train model
train(d_model, g_model, gan_model, dataset)

ValueError: in user code:

    c:\Users\probook\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    <ipython-input-485-99ab02836c17>:27 ploss  *
        feature_true = modelvgg(in_src)
    c:\Users\probook\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\engine\base_layer.py:1020 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    c:\Users\probook\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\engine\input_spec.py:269 assert_input_compatibility
        ', found shape=' + display_shape(x.shape))

    ValueError: Input 0 is incompatible with layer model: expected shape=(None, 256, 256, 3), found shape=(1, 16, 16, 1)
